In [1]:
%matplotlib inline
from __future__ import division
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy.random as npr
import glob
import scipy.stats as st
import statsmodels.api as sm


/home/peter/opt/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Summary

This notebook produces the results on mutation enrichment presented in Table S2.

In [2]:
# Load in the heteroplasmy data.
counts = pd.read_csv('table_s10_famfreqs.tsv', sep='\t', skiprows=1)
counts.head(2)

,fqid,position_rcrs,A,C,G,T,a,c,g,t,...,Sex,spike_in,haplogroup,fam_str,fam_cat,mot_cat,age_collection,age_birth,twin_info,heteroplasmy
0,M132-bl,14461,0,181,0,3895,0,170,0,4170,...,Female,0,J1c5d,0-0-1-2,m1c2,m1c2,16658,NaN,NaN,yes
1,M132-ch,14461,0,170,0,3327,0,147,0,3694,...,Female,0,J1c5d,0-0-1-2,m1c2,m1c2,16658,NaN,NaN,yes


In [3]:
# Load in the annotations
annot = pd.read_csv('table_s1_heteroplasmy_data.tsv', sep='\t', skiprows=2)

In [4]:
# sizes of the mt-genome in different categories
d_loop_len = 1122
trna_len = 1508
rrna_len = 2513
protein_s_len = 2834
protein_n_len = 8533
intergenic_len = 88
all_lens = np.array([d_loop_len, trna_len, rrna_len, protein_s_len, protein_n_len, intergenic_len])

In [5]:
# group by family-position combination
mut_annot = annot.groupby(['family_id', 'position_rcrs']).apply(lambda x: x.iloc[0,:])
mut_annot['gene'].replace(np.nan, 'intergenic', inplace=True)
tot_obs = mut_annot.shape[0]
assert tot_obs == 346

In [6]:
# the mutation positions
positions = mut_annot.index.get_level_values(1).values
positions[:5]

array([ 2377,  4332,  1282,  4520, 11088])

### D-loop

In [7]:
is_d_loop = lambda x: (16024 <= x) | (x <= 576)

In [8]:
num_d_loop = np.array(map(is_d_loop, positions)).sum()
num_d_loop

73

In [9]:
from __future__ import division
p_d_loop_expected = is_d_loop(np.arange(1,16570)).sum() / 16569
pval = st.binom_test(num_d_loop, tot_obs, p=p_d_loop_expected, alternative='two-sided')
print 'D-loop: observed, expected, binomial 2-sided p-value'
is_d_loop(positions).sum(), p_d_loop_expected*346, pval

D-loop: observed, expected, binomial 2-sided p-value


(73, 23.43001991671193, 4.0500484611955255e-18)

In [10]:
obs_trna = mut_annot['gene'].str.startswith('TR').sum()
p_trna_exp = 1508.0/16569
pval = st.binom_test(obs_trna, tot_obs, p_trna_exp, alternative='two-sided')
print 'tRNA: observed, expected, binomial 2-sided p-value'
print obs_trna, p_trna_exp*tot_obs, pval

tRNA: observed, expected, binomial 2-sided p-value
36 31.4906150039 0.3995571716659997


In [11]:
obs_rrna = mut_annot['gene'].str.startswith('RN').sum()
p_rrna_exp = 2513.0/16569
pval = st.binom_test(obs_rrna, tot_obs, p_trna_exp, alternative='two-sided')
print 'rRNA: observed, expected, binomial 2-sided p-value'
print obs_rrna, p_rrna_exp*tot_obs, pval

rRNA: observed, expected, binomial 2-sided p-value
42 52.4773975496 0.0606555594659883


In [12]:
num_syn = (mut_annot['syn'] == 'syn').sum()
p_syn_exp = 2834.0/16569
pval = st.binom_test(num_syn, tot_obs, p_syn_exp)
print 'syn: observed, expected, binomial 2-sided p-value'
print num_syn, p_syn_exp*tot_obs, pval

syn: observed, expected, binomial 2-sided p-value
76 59.1806385419 0.018445595490323933


In [13]:
num_nonsyn = (mut_annot['syn'] == 'nsyn').sum()
p_nsyn_exp = 8533/16569
pval = st.binom_test(num_nonsyn, tot_obs, p_nsyn_exp)
print 'nonsyn: observed, expected, binomial 2-sided p-value'
print num_nonsyn, p_nsyn_exp*tot_obs, pval

nonsyn: observed, expected, binomial 2-sided p-value
118 178.189269117 1.0142234592112752e-10


In [14]:
num_intergenic = (mut_annot['gene'] == 'intergenic').sum()  # == 1
p_inter_exp = 88.0/16569
pval = st.binom_test(num_intergenic, tot_obs, p_inter_exp)
print 'intergenic: observed, expected, binomial 2-sided p-value'
print num_intergenic, p_inter_exp*tot_obs, pval

intergenic: observed, expected, binomial 2-sided p-value
1 1.83764862092 1.0


In [15]:
is_coding = mut_annot['gene'].isin(['ATP6', 'ATP8', 'COX1', 'COX2', 'COX3', 'CYTB', 'ND1', 'ND2', 'ND3', 'ND4', 'ND5', 'ND6'])
num_coding_observed = is_coding.sum()

In [16]:
# This parses the features for genes.
# I downloaded features from NCBI GenBank rCRS entry:
# https://www.ncbi.nlm.nih.gov/nuccore/251831106
with open('features_rcrs.txt') as fin:
    features = np.array([re.sub('[A-Za-z()]', '', re.split(' +', el.strip())[-1]).split('..') for el in fin if 'CDS' in el]).astype(int)

In [17]:
features

array([[ 3307,  4262],
       [ 4470,  5511],
       [ 5904,  7445],
       [ 7586,  8269],
       [ 8366,  8572],
       [ 8527,  9207],
       [ 9207,  9990],
       [10059, 10404],
       [10470, 10766],
       [10760, 12137],
       [12337, 14148],
       [14149, 14673],
       [14747, 15887]])

In [18]:
num_coding_sites = np.diff(features, axis=1).sum()
p_coding_exp = num_coding_sites / 16569.0
num_coding_sites, p_coding_exp
num_coding_observed, p_coding_exp*346, st.binom_test(num_coding_observed, 346, p_coding_exp)

(194, 237.68314321926488, 9.595573572388399e-07)